# Part 1: Cleanup
### *Preparing raw data for modelling*

> #### by [Sebastian Einar Røkholt](https://www.linkedin.com/in/sebastianrokholt/)
>-------------------------------------------
> *January 2021*
----------------------------------------------------------------------------
<a id="top"></a> 
![Hybrid Recommender System Graphic](project-graphic.png)


### Project Index:
[GitHub Repo](http://url) <br>
*Part 1:* **cleanup.ipynb**<br>
*Part 2:* [analysis-and-modelling.ipynb](http://url)<br>
*Part 3:* [predict.ipynb](http://url)<br>
    
### Cleanup Notebook Index:
1. [**Cleaning 'rangering.dat': User ratings for movies**](#ratings) <br>
2. [**Cleaning 'bruker.json': Users data**](#users) <br>
3. [**Cleaning 'film.xlsx': Movies data**](#movies) <br>
----------------------------------------------------------------------------

## <a id="ratings"></a>1) Cleaning 'rangering.dat'-file: Preparing the user ratings dataset

In [1]:
# Importing packages for data cleanup
import pandas as pd
import time
import datetime
from math import ceil
from os import path, makedirs
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)

In [2]:
# Importing dataset as .csv, setting column labels
ratings_df = pd.read_csv('raw_data/rangering.dat', sep='::', header=0,
                         names=['BrukerID', 'FilmID', 'Rangering', 'Tidstempel'],
                         engine='python')

ratings_df.describe()

,BrukerID,FilmID,Rangering,Tidstempel
count,900187.000000,900187.000000,900187.000000,8.986950e+05
mean,2991.864495,1989.675878,4.279477,9.722414e+08
std,1736.204837,1126.366532,1.971075,1.214672e+07
min,0.000000,0.000000,1.000000,9.567039e+08
25%,1458.000000,1037.000000,3.000000,9.653029e+08
50%,2967.000000,1959.000000,4.000000,9.729904e+08
75%,4501.000000,2963.000000,5.000000,9.752202e+08
max,6040.000000,3952.000000,10.000000,1.046455e+09


In [3]:
# Check for missing values
missing_vals = ratings_df.isnull().sum()
print(missing_vals, '\n')
perc = round(missing_vals / ratings_df.shape[0] * 100, 2)
print(f'There are {ratings_df.shape[0]} rows in the dataset.')
print(f'Proportion of missing data for each column in %: \n{perc}')

BrukerID         0
FilmID           0
Rangering        0
Tidstempel    1492
dtype: int64 

There are 900187 rows in the dataset.
Proportion of missing data for each column in %: 
BrukerID      0.00
FilmID        0.00
Rangering     0.00
Tidstempel    0.17
dtype: float64


There are almost 1500 rows with missing timestamp values, which culminates to only 0.17% of the dataset. Even though 0.17% missing data is low, I don't want to simply delete these rows, as it would decrease the sample size. This would  result in a significant decrease in prediction accuracy for a large number of users. I should therefore try to fill these missing values by imputing an average. However, it would not make sense to impute the average of all timestamps, as this would tamper with calculations of how long a user has been active on the platform. A better solution is to impute the average timestamp for each individual user. 

In [4]:
missing_df = ratings_df[ratings_df['Tidstempel'].isnull()]
not_missing_df = ratings_df[ratings_df['Tidstempel'].notnull()]

# Store the timestamp of the date of conversion (965088000) as a variable
date = '01/08/2000 01:00:00'  # UTC +1
date_of_conversion = time.mktime(datetime.datetime.strptime(date, "%d/%m/%Y %H:%M:%S").utctimetuple())
print('date_of_conversion', date_of_conversion)  # 965088000

old_scaling = not_missing_df[not_missing_df['Tidstempel'] < date_of_conversion]
new_scaling = not_missing_df[not_missing_df['Tidstempel'] >= date_of_conversion]

rated_before = old_scaling['BrukerID'].unique().tolist()
rated_after = new_scaling['BrukerID'].unique().tolist()
rated_before_and_after = []

for user in rated_before:
    if user in rated_after:
        rated_before_and_after.append(user)
# print('RATED BOTH:', len(rated_before_and_after))   # 383 user rated both before and after

rated_only_before = sorted(list(set(rated_before).difference(rated_before_and_after)))
rated_only_after = sorted(list(set(rated_after).difference(rated_before_and_after)))

imputed = 0
for row in missing_df.iterrows():
    index = row[0]
    user_id = int(row[1][0])

    if user_id in rated_only_before:
        avg_timestamp = old_scaling.loc[old_scaling['BrukerID'] == user_id, 'Tidstempel'].mean()
        ratings_df.loc[index, 'Tidstempel'] = avg_timestamp
        imputed += 1

    elif user_id in rated_only_after:
        avg_timestamp = new_scaling.loc[new_scaling['BrukerID'] == user_id, 'Tidstempel'].mean()
        ratings_df.loc[index, 'Tidstempel'] = avg_timestamp
        imputed += 1

print(f'Total number of missing values after imputing the avg timestamp for {imputed} entries: ',
      ratings_df.isnull().sum().sum(), '\n')

# Now we can determine if there are any of the 131 remaining ratings with a rating > 5:
missing_df = ratings_df[ratings_df['Tidstempel'].isnull()]
missing_and_high_rating = missing_df.loc[missing_df['Rangering'] > 5]
print('missing timestamp and high rating: ', len(missing_and_high_rating))

# Imputing the average timestamp for old ratings for the 63 entries with missing timestamp and a rating > 5
for row in missing_and_high_rating.iterrows():
    index = row[0]
    user_id = int(row[1][0])
    avg_timestamp = old_scaling.loc[:, 'Tidstempel'].mean()
    ratings_df.loc[index, 'Tidstempel'] = avg_timestamp

# Dropping remaining 68 entries with missing timestamp values and a rating < 6
ratings_df.dropna(how='any', inplace=True)

print(f'Missing values after deleting the remaining entries with a rating < 6: {ratings_df.isnull().sum().sum()}')


# Converting the scale for ratings prior to 01.08.2000 from 1-10 to 1-5.
# First, converting the timestamp from float to int
ratings_df['Tidstempel'] = ratings_df['Tidstempel'].astype(int)

# Splitting the dataset into pre and post conversion date again,
# this time converting the old ratings to the new scale.
old_scaling = ratings_df[ratings_df['Tidstempel'] < date_of_conversion]
new_scaling = ratings_df[ratings_df['Tidstempel'] >= date_of_conversion]

pd.reset_option('mode.chained_assignment')  # THE CODE BELOW CAUSES SETTINGWITHCOPYWARNING, DUE TO CHAINED ASSIGNMENT
with pd.option_context('mode.chained_assignment', None):
    # Converting the scale for ratings from 1-10 to 1-5.
    # Replacing the values from integers to strings, so that I can use regex on them
    old_scaling['Rangering'].replace(to_replace=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                                     value=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], inplace=True)

    # Using regex to find values and then replacing them with corresponding values on the new scale.
    # 1's and 2's = 1, 3's and 4's = 2, etc.
    old_scaling['Rangering'].replace(regex=True, to_replace=['1\b|2', '3|4', '5|6', '7|8', '9|10'],
                                     value=[1, 2, 3, 4, 5], inplace=True)

    # Appending the new ratings to the old
    ratings_df_cleaned = old_scaling.append(new_scaling, ignore_index=True)

date_of_conversion 965088000.0
Total number of missing values after imputing the avg timestamp for 1359 entries:  133 

missing timestamp and high rating:  63
Missing values after deleting the remaining entries with a rating < 6: 0


In [5]:
# Creating a new folder for the cleaned data, unless it already exists
if path.exists('cleaned_data'):
    print('cleaned_data folder already exists')
else:
    makedirs('cleaned_data')

# Writing the dataframe to a .csv file and storing it in cleaned data
ratings_df_cleaned.to_csv('cleaned_data/rangering.csv', index=False)

cleaned_data folder already exists


------------------------------------------------------------------------------
## <a id="users"></a>2) Cleaning 'bruker.json' for user dataset

In [6]:
# Importing dataset as DataFrame object, setting column labels
users_df = pd.read_json('raw_data/bruker.json', orient='split')

users_df.head(5)

,BrukerID,Kjonn,Alder,Jobb,Postkode
0,0,None,45.0,6.0,92103
1,1,M,50.0,16.0,55405-2546
2,2,M,18.0,20.0,44089
3,3,M,NaN,1.0,33304
4,4,M,35.0,6.0,48105


In [7]:
# Describe dataset
users_df.describe()

,BrukerID,Alder,Jobb
count,6040.000000,5046.000000,5447.000000
mean,3020.465894,30.666072,9.104278
std,1743.799216,12.954723,11.239708
min,0.000000,1.000000,0.000000
25%,1510.750000,25.000000,3.000000
50%,3020.500000,25.000000,7.000000
75%,4530.250000,35.000000,14.000000
max,6040.000000,56.000000,99.000000


In [8]:
# Check for missing values
missing_vals = users_df.isnull().sum()
perc = round(missing_vals / users_df.shape[0] * 100, 2)
print(f'There are {users_df.shape[0]} rows in the dataset.')
print(f'Proportion of missing data for each column in %: \n{perc}')

There are 6040 rows in the dataset.
Proportion of missing data for each column in %: 
BrukerID     0.00
Kjonn        5.02
Alder       16.46
Jobb         9.82
Postkode     7.47
dtype: float64


16.5% of the userbase has not recorded their age. This is a significant proportion, so I will have to be careful when handling these missing values. 

In [9]:
# Count rows that have at least N missing values
def count_rows_with_n_missing_vals(dataframe, n):
    missing_val_count = dataframe.shape[0] - (dataframe.dropna(how='any', thresh=(dataframe.shape[1] - n)+1).shape[0])
    return missing_val_count


print(f'There are {count_rows_with_n_missing_vals(users_df, 2)} rows with at least 2 missing values')
print(f'There are {count_rows_with_n_missing_vals(users_df, 3)} rows with at least 3 missing values')

There are 291 rows with at least 2 missing values
There are 20 rows with at least 3 missing values


There are a few records (20) that have most of the data missing, and there are almost 300 records that are missing 2 values or more. However, since these users are likely to have recommended movies and that their recommendations are recorded in the ratings dataset, I can't just remove these values altogether. I am also restricted by the fact that training a model with a dataset that has a lot of missing values might significantly impact its accuracy. User-based filtering algorithms assume that all values are numerical and that they hold a meaningful value.

I can conclude that these missing values need to be filled, but I should be wary of just imputing a value of 0 or an average or median value. The features in the user dataset are somewhat correlated, and this correlation is important for my recommendation system to utilize. Luckily, there are some features that are more suited for constant imputation than the rest.

#### Imputation for missing data in users dataset: Constant

Imputing a 0 to fill missing values in the 'Jobb' -column is accurate, as this label indicates 'other/unspecified' according to the README for this dataset. Also, since we are dealing with a continuous variable that is not related to any other independent variable, imputing a constant value shouldn't lead to a significant loss inefficiency. Executing this step before doing any further imputations might benefit a predictive modeling approach to imputing the rest of the missing values.

In [10]:
# Fill NA values in 'Jobb' column with 0, indicating 'other/unspecified'
users_df['Jobb'].fillna(0, inplace=True)
users_df.head()

,BrukerID,Kjonn,Alder,Jobb,Postkode
0,0,None,45.0,6.0,92103
1,1,M,50.0,16.0,55405-2546
2,2,M,18.0,20.0,44089
3,3,M,NaN,1.0,33304
4,4,M,35.0,6.0,48105


#### Imputation for missing data in users dataset: KNN

The remainding columns with missing values are 'Postkode' (postal code/zip code), 'Kjonn' (gender) and 'Alder'(age). For these features I will use KNN imputation to handle the missing data. The KNN imputation algorithm will find predict a new sample to replace the missing value in the target row by finding the N (5 in this case) most similar rows to the target and then calculating the average of these. These similar rows don't have any missing data (at least not for the target feature), so the predicted value can then be inserted into the target row. 

Using KNN will be more accurate than taking a simple average, but though age, gender, field of work and address may be somewhat related, the correlation is not likely to be very strong. However, KNN algorihms are very sensitive to outliers, so the spread of the dataset might increase, which might make it easier to make recommendations. 

In [11]:
# PREPARING DATA FOR KNN IMPUTER
# 1) Normalizing gender categories from 'M' and 'F' to 1 and 0
users_df.replace(to_replace=['M', 'F'], value=[1, 0], inplace=True)

# 2) Adding the postal codes without the hyphen and the following integers with Sector, Blocks, and Side of Street
#    from postal codes to a new column, which can be used for KNN predictions
users_df['Postkode_5'] = users_df.Postkode.str[:3]

# 3) Removing features that are unecessary/unhelpful for KNN predictions
df = users_df.drop(['Postkode', 'BrukerID'], axis=1)

# 4) Scaling the numeric variables to have values between 0 and 1.
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)


In [12]:
# Perform KNN imputation on missing values
imputer = KNNImputer(n_neighbors=5, )
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
print(df.isna().sum())

Kjonn         0
Alder         0
Jobb          0
Postkode_5    0
dtype: int64


In [13]:
# Reverting scaling, so that the data is readable
df = pd.DataFrame(scaler.inverse_transform(df), columns=df.columns)

# Adding 'BrukerID' and complete postal codes back again
df[['BrukerID', 'Postkode']] = users_df[['BrukerID', 'Postkode']]

# Renaming the new dataframe in order to replace the old one, while reordering the columns simultaenously
users_df = df[['BrukerID', 'Kjonn', 'Alder', 'Jobb', 'Postkode', 'Postkode_5']]

# Iterating over the dataframe, fixing stuff as I go
for row in users_df.iterrows():
    index = row[0]
    gender = row[1][1]
    age = row[1][2]
    job = row[1][3]
    postcode = str(row[1][4])
    postcode_5 = row[1][5]

    # Rounding imputed postal code values from float to whole int. Then convert to string
    users_df.iloc[index, 5] = str(int(round(postcode_5)))

    # Inserting the full postal codes back into the new postal code column.
    # This leaves the imputed postcode values as three-digit postal codes,
    # which means that they can't be used to send anything by mail. However, they can be used for
    # user-based filtering or profiling, as a postal code with three digits indicates the region/city
    # in which the user most likely would reside in.
    if len(postcode) >= 5:
        users_df.iloc[index, 5] = postcode

    # Rounding/normalizing gender values from float to either 'M' if >=0.5 or else 'F'
    if gender >= 0.5:
        users_df.iloc[index, 1] = 'M'
    else:
        users_df.iloc[index, 1] = 'F'

    # Rounding age values to closest age interval according to the README
    if age < 18:
        users_df.iloc[index, 2] = 1
    elif 18 <= age < 25:
        users_df.iloc[index, 2] = 18
    elif 25 <= age < 35:
        users_df.iloc[index, 2] = 25
    elif 35 <= age < 45:
        users_df.iloc[index, 2] = 35
    elif 45 <= age < 50:
        users_df.iloc[index, 2] = 45
    elif 50 <= age < 56:
        users_df.iloc[index, 2] = 50
    else:
        users_df.iloc[index, 2] = 56

# Changing features that are dtype float (Kjonn, Alder, Jobb) back into dtype int
users_df[['Alder', 'Jobb']] = users_df[['Alder', 'Jobb']].astype(dtype=int)

# Remove the old postal code column, rename the new one
users_df['Postkode'] = users_df['Postkode_5']
users_df.drop('Postkode_5', axis=1, inplace=True)

print(users_df.head(20))

    BrukerID Kjonn  Alder  Jobb    Postkode
0          0     F     45     6       92103
1          1     M     50    16  55405-2546
2          2     M     18    20       44089
3          3     M     35     1       33304
4          4     M     35     6       48105
5          5     M     25    20         664
6          6     M     50    14         379
7          7     F     25     0         264
8          8     M     25     4       70806
9          9     M     25    19       45701
10        10     F     18     1       95864
11        11     M     35     1         478
12        12     M     45     0       10543
13        13     M     50     7       34243
14        14     M     25     4       53140
15        15     F     18     4       60625
16        16     M     25    17       03570
17        17     M     35     7       30117
18        18     M     50     1       01096
19        19     M     25    15       02143


In [14]:
# Check for missing values again
missing_vals = users_df.isnull().sum()
perc = round(missing_vals / users_df.shape[0] * 100, 2)
print(f'There are {users_df.shape[0]} rows in the dataset.')
print(f'Proportion of missing data for each column in %: \n{perc}')

There are 6040 rows in the dataset.
Proportion of missing data for each column in %: 
BrukerID    0.0
Kjonn       0.0
Alder       0.0
Jobb        0.0
Postkode    0.0
dtype: float64


In [15]:
# Writing the dataframe to a .csv file and storing it in cleaned data folder
users_df.to_csv('cleaned_data/bruker.csv', index=False)

------------------------------------------------------------------------------
## <a id="movies"></a>3) Cleaning 'film.xlsx': Preparing the movies dataset

In [16]:
# Load data from Excel file to DataFrame object
excel = pd.ExcelFile('raw_data/film.xlsx')
movies_df = excel.parse(sheet_name='film', index_col=None)
movies_df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
print(f'There are {movies_df.count()[0]} movies in the dataset')

There are 3883 movies in the dataset


In [17]:
# Check for missing values
missing_vals = movies_df.isnull().sum()
perc = round(missing_vals / movies_df.shape[0] * 100, 2)
print(f'Proportion of missing data for each column in %: \n{perc}')

# Check for duplicate rows
duplicate_rows = movies_df[movies_df.duplicated(['FilmID'])]
print(duplicate_rows)
duplicate_rows = movies_df[movies_df.duplicated(['Tittel'])]
print(duplicate_rows)

Proportion of missing data for each column in %: 
FilmID     0.0
Tittel     0.0
Sjanger    0.0
dtype: float64
Empty DataFrame
Columns: [FilmID, Tittel, Sjanger]
Index: []
Empty DataFrame
Columns: [FilmID, Tittel, Sjanger]
Index: []


There are no missing or duplicate values in this dataset.

In [18]:
# CONVERTING GENRE FEATURE FROM ONE NOMINAL COLUMN TO 18 DISCRETE COLUMNS
# 1) Adding labels for the 18 different genres with 0 as default value
genres = ["Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
          "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
for genre in genres:
    movies_df.loc[:,genre] = 0


In [19]:
# 2) Looping over the nominal genre column, and setting the discrete columns accordingly
for row in movies_df.iterrows():
    index = row[0]
    genre_data = row[1][2]

    genres = movies_df.columns.values.tolist()
    genres.remove('FilmID')
    genres.remove('Tittel')
    genres.remove('Sjanger')

    for genre in genres:
        if genre in genre_data:
            movies_df.loc[index, genre] = 1

In [20]:
# Remove genre nominal column, as it is no longer needed
movies_df.drop(labels=['Sjanger'], axis=1, inplace=True)

In [21]:
# Writing the dataframe to a .csv file and storing it in cleaned data folder
movies_df.to_csv('cleaned_data/film.csv', index=False)